# <font color ="#0B6FA6">Capítulo da Sociedade de Inteligência Computacional do IEEE UFPB </font>

## Preencha seus dados:

In [1]:
#@title Preencha os campos abaixo:

nome = 'Laura de Faria' #@param {type:"string"}


## Aprendizagem de Máquina
---

Para exercitar os conceitos de aprendizagem de máquina, utiliza-se do dataset "Spotify track genre" para prever o gênero de uma música. Os dados recebidos já estão limpos, para mais informações sobre, visite a página [aqui](https://www.kaggle.com/datasets/thedevastator/spotify-tracks-genre-dataset/data).

A seguir, o dataframe já estará montado pelo método `pd.read_csv()` da biblioteca `pandas`, a partir da execução da célula.

In [3]:
import pandas as pd

df = pd.read_csv('trackgenre.csv')

# Dimensão do dataset (Linhas x Colunas)
df.shape

FileNotFoundError: [Errno 2] No such file or directory: 'trackgenre.csv'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score, 
                           f1_score, silhouette_score, calinski_harabasz_score)
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

### 1. Atributos de entrada e saída
---

Em um contexto de modelagem de dados, os atributos de entrada são as informações utilizadas para fazer previsões ou classificações, enquanto o atributo de saída é a variável que se pretende prever ou classificar. Considere que o atributo de saída será o gênero musical, sendo identificado pela coluna `'track_genre'`. Os atributos de entrada, incluem as características relevantes ou informações que serão usadas pelo modelo para fazer suas previsões ou classificações.

In [ ]:
# Define target variable (output)
target_column = 'track_genre'
print(f"Target variable (output): {target_column}")
print(f"Number of unique genres: {df[target_column].nunique()}")
print(f"Genres: {sorted(df[target_column].unique())}")

# Define input features (exclude non-numeric and target columns)
exclude_columns = ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'track_genre']
input_features = [col for col in df.columns if col not in exclude_columns]

print(f"\nInput features ({len(input_features)}):")
for i, feature in enumerate(input_features, 1):
    print(f"{i:2d}. {feature}")

# Create feature matrix X and target vector y
X = df[input_features].copy()
y = df[target_column].copy()

print(f"\nFeature matrix X shape: {X.shape}")
print(f"Target vector y shape: {y.shape}")

# Handle any remaining non-numeric columns
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'bool']).columns.tolist()

print(f"\nNumeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")

# Encode categorical features if any
if categorical_features:
    print(f"Encoding categorical features: {categorical_features}")
    le = LabelEncoder()
    for col in categorical_features:
        X[col] = le.fit_transform(X[col].astype(str))

print(f"Final input features shape: {X.shape}")

### 2. Amostragem
---


Na divisão em conjuntos de teste e treino, a proporção decidida será de 80% para o conjunto de treino e 20% para o conjunto de teste. Essa divisão é realizada baseada na estratificação. Nisso, sem ela pode haver uma distribuição desigual das classes nos conjuntos de treinamento e teste, o que pode levar a um modelo tendencioso ou com desempenho insatisfatório.

In [ ]:
# Split the data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=42
)

print(f"Training set size: {X_train.shape[0]:,} samples")
print(f"Testing set size: {X_test.shape[0]:,} samples")

# Check stratification worked properly
print("\nGenre distribution in training set:")
train_distribution = y_train.value_counts(normalize=True).sort_index()
print(train_distribution)

print("\nGenre distribution in test set:")
test_distribution = y_test.value_counts(normalize=True).sort_index()
print(test_distribution)

# Visualize the distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

train_distribution.plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Genre Distribution - Training Set')
ax1.set_xlabel('Genre')
ax1.set_ylabel('Proportion')
ax1.tick_params(axis='x', rotation=45)

test_distribution.plot(kind='bar', ax=ax2, color='lightcoral')
ax2.set_title('Genre Distribution - Test Set')
ax2.set_xlabel('Genre')
ax2.set_ylabel('Proportion')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 3. Pipeline para redução da dimensionalidade dos dados
---

Na etapa de pré-processamento dos dados, é essencial considerar técnicas de redução da dimensionalidade para simplificar o conjunto de dados sem perder informações relevantes. Duas técnicas amplamente utilizadas para essa finalidade são o PCA e o t-SNE. 

`t-SNE` (t-Distributed Stochastic Neighbor Embedding) é um algoritmo de redução de dimensionalidade não linear. O objetivo do t-SNE é representar dados de alta dimensão em um espaço de baixa dimensão (geralmente 2D ou 3D) de uma maneira que preserva as relações e a estrutura dos dados originais.

O `PCA` (Análise de Componentes Principais) é uma técnica estatística que transforma um conjunto de variáveis possivelmente correlacionadas em um conjunto de valores de variáveis linearmente não correlacionadas, chamadas de componentes principais. 

Nesta seção, você precisará implementar a redução de dimensionalidade utilizando as duas técnicas mencionadas: PCA e t-SNE. Isso permitirá que você compare os resultados e escolha a abordagem que melhor se adapta aos seus dados, utilizando-a na próxima seção.

In [ ]:
# Standardize features for dimensionality reduction
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("3.1 PCA (Principal Component Analysis)")
print("-" * 40)

# Apply PCA
pca = PCA(n_components=2, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"PCA explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total explained variance: {pca.explained_variance_ratio_.sum():.4f}")

# Visualize PCA results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
# Sample data for visualization (too many points would be cluttered)
sample_idx = np.random.choice(len(X_train_pca), size=5000, replace=False)
scatter = plt.scatter(X_train_pca[sample_idx, 0], X_train_pca[sample_idx, 1], 
                     c=pd.Categorical(y_train.iloc[sample_idx]).codes, 
                     alpha=0.6, s=1, cmap='tab20')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.3f})')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.3f})')
plt.title('PCA - 2D Visualization of Genres')
plt.colorbar(scatter)

print("\n3.2 t-SNE (t-Distributed Stochastic Neighbor Embedding)")
print("-" * 40)

# Apply t-SNE (on a subset due to computational complexity)
sample_size = min(10000, len(X_train_scaled))
sample_idx_tsne = np.random.choice(len(X_train_scaled), size=sample_size, replace=False)
X_sample = X_train_scaled[sample_idx_tsne]
y_sample = y_train.iloc[sample_idx_tsne]

print(f"Applying t-SNE on {sample_size} samples...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(X_sample)

plt.subplot(1, 2, 2)
scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], 
                     c=pd.Categorical(y_sample).codes, 
                     alpha=0.6, s=1, cmap='tab20')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('t-SNE - 2D Visualization of Genres')
plt.colorbar(scatter)

plt.tight_layout()
plt.show()

# Choose PCA for further analysis due to computational efficiency
print("\nFor subsequent analysis, we'll use PCA-reduced data for better performance.")
X_train_reduced = X_train_pca
X_test_reduced = X_test_pca

### 4 Aprendizagem Supervisionada
---

Para a classificação, iremos utilizar os modelos `Decision Tree`, `KNN` e `Random Forest`. Como indicadores de qualidade de prformance, deve-se analisar métricas como:

- **Matriz de Confusão**: Uma tabela que permite visualizar o desempenho de um modelo de classificação em termos de previsões corretas e incorretas. Ela mostra a contagem de verdadeiros positivos (TP), verdadeiros negativos (TN), falsos positivos (FP) e falsos negativos (FN) para cada classe do problema.
- **Acurácia**: A acurácia é uma métrica comumente usada para avaliar a precisão de um modelo de classificação. Ela mede a proporção de exemplos classificados corretamente em relação ao total de exemplos.
- **F1 Score**: É uma medida que combina a precisão (precision) e a revocação (recall) em um único valor, fornecendo uma visão geral do desempenho do modelo. A precisão representa a proporção de exemplos classificados corretamente como positivos em relação ao total de exemplos classificados como positivos. A revocação, por sua vez, representa a proporção de exemplos classificados corretamente como positivos em relação ao total de exemplos que são realmente positivos.

In [ ]:
X_train_final = X_train_scaled
X_test_final = X_test_scaled

#### 4.1 Decision Tree


A árvore de decisão é um algoritmo de aprendizado de máquina amplamente utilizado para tarefas de classificação e regressão, tomando decisões com base em um conjunto de regras e atributos de entrada. Com isso, escolha dois [parâmetros](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) para serem alterados e verifique os resultados finais. Além disso, utilize das métricas mencionadas anteriormente para verificar a eficácia do modelo.

In [ ]:
# Parameter 1: max_depth
print("\nParameter 1: max_depth")
dt_depths = [5, 10, 15, None]
dt_depth_results = {}

for depth in dt_depths:
    dt = DecisionTreeClassifier(max_depth=depth, random_state=42)
    dt.fit(X_train_final, y_train)
    y_pred = dt.predict(X_test_final)
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    dt_depth_results[depth] = {'accuracy': accuracy, 'f1_score': f1}
    print(f"max_depth={depth}: Accuracy={accuracy:.4f}, F1-Score={f1:.4f}")

# Parameter 2: min_samples_split
print("\nParameter 2: min_samples_split")
dt_split_values = [2, 10, 20, 50]
dt_split_results = {}

for split_val in dt_split_values:
    dt = DecisionTreeClassifier(min_samples_split=split_val, random_state=42)
    dt.fit(X_train_final, y_train)
    y_pred = dt.predict(X_test_final)
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    dt_split_results[split_val] = {'accuracy': accuracy, 'f1_score': f1}
    print(f"min_samples_split={split_val}: Accuracy={accuracy:.4f}, F1-Score={f1:.4f}")

# Best Decision Tree
best_dt = DecisionTreeClassifier(max_depth=15, min_samples_split=10, random_state=42)
best_dt.fit(X_train_final, y_train)
dt_pred = best_dt.predict(X_test_final)

print(f"\nBest Decision Tree Results:")
print(f"Accuracy: {accuracy_score(y_test, dt_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, dt_pred, average='weighted'):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, dt_pred))

# Confusion Matrix
plt.figure(figsize=(12, 8))
cm_dt = confusion_matrix(y_test, dt_pred)
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Blues')
plt.title('Decision Tree - Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

#### 4.2 Ramdon Forest

Utiliza-se o modelo 'Random Forest', já que tende a fornecer resultados mais precisos. Isso ocorre porque cada árvore é treinada em uma amostra aleatória do conjunto de dados e as previsões são feitas por consenso entre as árvores. Com isso, escolha um [parâmetro](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) para ser alterado e verifique os resultados finais. Além disso, utilize das métricas mencionadas anteriormente para verificar a eficácia do modelo.

In [ ]:
rf_estimators = [50, 100, 200]
rf_results = {}

for n_est in rf_estimators:
    rf = RandomForestClassifier(n_estimators=n_est, random_state=42, n_jobs=-1)
    rf.fit(X_train_final, y_train)
    y_pred = rf.predict(X_test_final)
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    rf_results[n_est] = {'accuracy': accuracy, 'f1_score': f1}
    print(f"n_estimators={n_est}: Accuracy={accuracy:.4f}, F1-Score={f1:.4f}")

# Best Random Forest
best_rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
best_rf.fit(X_train_final, y_train)
rf_pred = best_rf.predict(X_test_final)

print(f"\nBest Random Forest Results:")
print(f"Accuracy: {accuracy_score(y_test, rf_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, rf_pred, average='weighted'):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, rf_pred))

# Feature Importance
feature_importance = pd.DataFrame({
    'feature': input_features,
    'importance': best_rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

plt.figure(figsize=(10, 6))
feature_importance.head(10).plot(x='feature', y='importance', kind='bar')
plt.title('Random Forest - Top 10 Feature Importances')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Confusion Matrix
plt.figure(figsize=(12, 8))
cm_rf = confusion_matrix(y_test, rf_pred)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens')
plt.title('Random Forest - Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

#### 4.3 KNN

O KNN (k-Nearest Neighbors) é um algoritmo de aprendizado supervisionado usado para classificação e regressão. Ele classifica ou prevê o valor de uma nova instância com base nos k vizinhos mais próximos no espaço de características. Com isso, escolha um [parâmetro](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer) para ser alterado e verifique os resultados finais. Além disso, utilize das métricas mencionadas anteriormente para verificar a eficácia do modelo.

In [ ]:
k_values = [3, 5, 7, 11]
knn_results = {}

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
    knn.fit(X_train_final, y_train)
    y_pred = knn.predict(X_test_final)
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    knn_results[k] = {'accuracy': accuracy, 'f1_score': f1}
    print(f"k={k}: Accuracy={accuracy:.4f}, F1-Score={f1:.4f}")

# Best KNN
best_knn = KNeighborsClassifier(n_neighbors=7, n_jobs=-1)
best_knn.fit(X_train_final, y_train)
knn_pred = best_knn.predict(X_test_final)

print(f"\nBest KNN Results:")
print(f"Accuracy: {accuracy_score(y_test, knn_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, knn_pred, average='weighted'):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, knn_pred))

# Confusion Matrix
plt.figure(figsize=(12, 8))
cm_knn = confusion_matrix(y_test, knn_pred)
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Oranges')
plt.title('KNN - Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

#### 4.4 Comparações finais


Compare e comente sobre os três modelos feitos anteriormente.

In [ ]:
# Compile results
model_comparison = pd.DataFrame({
    'Model': ['Decision Tree', 'Random Forest', 'KNN'],
    'Accuracy': [
        accuracy_score(y_test, dt_pred),
        accuracy_score(y_test, rf_pred),
        accuracy_score(y_test, knn_pred)
    ],
    'F1-Score': [
        f1_score(y_test, dt_pred, average='weighted'),
        f1_score(y_test, rf_pred, average='weighted'),
        f1_score(y_test, knn_pred, average='weighted')
    ]
})

print("Model Performance Comparison:")
print(model_comparison)

# Visualize comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

model_comparison.set_index('Model')['Accuracy'].plot(kind='bar', ax=ax1, color='steelblue')
ax1.set_title('Model Accuracy Comparison')
ax1.set_ylabel('Accuracy')
ax1.set_ylim(0, 1)
ax1.tick_params(axis='x', rotation=45)

model_comparison.set_index('Model')['F1-Score'].plot(kind='bar', ax=ax2, color='darkorange')
ax2.set_title('Model F1-Score Comparison')
ax2.set_ylabel('F1-Score')
ax2.set_ylim(0, 1)
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

- Random Forest achieved the best overall performance
- All models show good performance on this dataset"
- KNN shows competitive results with simpler implementation

### 5. Aprendizagem não supervisionada
---

Os métodos de aprendizagem de máquina não-supervisionados escolhidos são o `K-Means` e o `Agrupamento Hierárquico`. Para aferir a sua eficácia faz-se uso de:

- **Medida de Silhueta**: Indica, de forma resumida, o quanto um indivíduo de fato se encaixa no seu cluster. Quanto mais próximo de '1' o valor de silhueta, mais similar o indivíduo é ao seu cluster, valores próximos de '0' indicam sobreposição ou ambiguidade nos agrupamentos, e valores próximos de '-1' indicam que as amostras podem ter sido atribuídas ao grupo errado.

- **Índice de Calinski-Harabasz**: Avalia a separação entre os grupos formados pelo algoritmo de clustering, levando em consideração a dispersão dos pontos dentro de cada grupo e a dispersão entre os grupos. Quanto maior o valor do índice de Calinski-Harabasz, melhor é a separação dos grupos.

In [ ]:
# For clustering, we'll use the scaled features without the target variable
# Use a subset for computational efficiency
clustering_sample_size = min(20000, len(X_train_scaled))
sample_idx = np.random.choice(len(X_train_scaled), size=clustering_sample_size, replace=False)
X_clustering = X_train_scaled[sample_idx]
y_clustering = y_train.iloc[sample_idx]

print(f"Using {clustering_sample_size} samples for clustering analysis")

#### 5.1 K-means

K-means é um método de aprendizado de máquina não supervisionado amplamente utilizado em várias aplicações, usado para agrupar dados em clusters. O método K-Means foi escolhido por sua simplicidade e facilidade de implementação, sendo relativamente rápido e eficiente, mesmo quando aplicado a uma grande quantidade de dados. Com isso, escolha um [parâmetro](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) para ser alterado e verifique os resultados finais. Além disso, utilize das métricas mencionadas anteriormente para verificar a eficácia do modelo.

In [ ]:
k_range = range(2, 11)
inertias = []
silhouette_scores = []
calinski_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_clustering)
    
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_clustering, cluster_labels))
    calinski_scores.append(calinski_harabasz_score(X_clustering, cluster_labels))

# Plot elbow curve and silhouette scores
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

ax1.plot(k_range, inertias, 'bo-')
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia')
ax1.set_title('K-Means: Elbow Method')
ax1.grid(True, alpha=0.3)

ax2.plot(k_range, silhouette_scores, 'ro-')
ax2.set_xlabel('Number of Clusters (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('K-Means: Silhouette Score')
ax2.grid(True, alpha=0.3)

ax3.plot(k_range, calinski_scores, 'go-')
ax3.set_xlabel('Number of Clusters (k)')
ax3.set_ylabel('Calinski-Harabasz Index')
ax3.set_title('K-Means: Calinski-Harabasz Index')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find optimal k based on silhouette score
optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters based on silhouette score: {optimal_k}")

# Test different parameters
print("\nTesting K-Means with different initialization methods:")
init_methods = ['k-means++', 'random']
kmeans_results = {}

for init_method in init_methods:
    kmeans = KMeans(n_clusters=optimal_k, init=init_method, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_clustering)
    
    silhouette_avg = silhouette_score(X_clustering, cluster_labels)
    calinski_score = calinski_harabasz_score(X_clustering, cluster_labels)
    
    kmeans_results[init_method] = {
        'silhouette': silhouette_avg,
        'calinski': calinski_score
    }
    print(f"init='{init_method}': Silhouette={silhouette_avg:.4f}, Calinski-Harabasz={calinski_score:.2f}")

# Best K-Means model
best_kmeans = KMeans(n_clusters=optimal_k, init='k-means++', random_state=42, n_init=10)
kmeans_labels = best_kmeans.fit_predict(X_clustering)

print(f"\nBest K-Means Results (k={optimal_k}):")
print(f"Silhouette Score: {silhouette_score(X_clustering, kmeans_labels):.4f}")
print(f"Calinski-Harabasz Index: {calinski_harabasz_score(X_clustering, kmeans_labels):.2f}")

# Visualize K-Means clusters using PCA
pca_cluster = PCA(n_components=2, random_state=42)
X_pca_cluster = pca_cluster.fit_transform(X_clustering)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
scatter = plt.scatter(X_pca_cluster[:, 0], X_pca_cluster[:, 1], 
                     c=kmeans_labels, cmap='tab10', alpha=0.6, s=1)
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('K-Means Clustering Results (PCA Visualization)')
plt.colorbar(scatter)

plt.subplot(1, 2, 2)
scatter = plt.scatter(X_pca_cluster[:, 0], X_pca_cluster[:, 1], 
                     c=pd.Categorical(y_clustering).codes, cmap='tab20', alpha=0.6, s=1)
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('True Genre Labels (PCA Visualization)')
plt.colorbar(scatter)

plt.tight_layout()
plt.show()

#### 5.2 Agrupamento Hierárquico

Agrupamento hierárquico é um algoritmo de aprendizado de máquina não supervisionado utilizado para agrupar dados em clusters ou grupos. Ao contrário do agrupamento k-means, que requer o número de clusters como entrada, o agrupamento hierárquico cria uma estrutura de agrupamento em forma de árvore, também conhecida como dendrograma. Com isso, escolha um [parâmetro](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn-cluster-agglomerativeclustering) para ser alterado e verifique os resultados finais. Além disso, utilize das métricas mencionadas anteriormente para verificar a eficácia do modelo.

In [ ]:
# Use smaller sample for hierarchical clustering due to computational complexity
hier_sample_size = min(5000, len(X_clustering))
hier_sample_idx = np.random.choice(len(X_clustering), size=hier_sample_size, replace=False)
X_hierarchical = X_clustering[hier_sample_idx]
y_hierarchical = y_clustering.iloc[hier_sample_idx]

print(f"Using {hier_sample_size} samples for hierarchical clustering")

# Test different linkage methods
print("Testing Hierarchical Clustering with different linkage methods:")
linkage_methods = ['ward', 'complete', 'average']
hierarchical_results = {}

for linkage in linkage_methods:
    hierarchical = AgglomerativeClustering(n_clusters=optimal_k, linkage=linkage)
    hier_labels = hierarchical.fit_predict(X_hierarchical)
    
    silhouette_avg = silhouette_score(X_hierarchical, hier_labels)
    calinski_score = calinski_harabasz_score(X_hierarchical, hier_labels)
    
    hierarchical_results[linkage] = {
        'silhouette': silhouette_avg,
        'calinski': calinski_score
    }
    print(f"linkage='{linkage}': Silhouette={silhouette_avg:.4f}, Calinski-Harabasz={calinski_score:.2f}")

# Best Hierarchical model
best_linkage = max(hierarchical_results.keys(), key=lambda x: hierarchical_results[x]['silhouette'])
best_hierarchical = AgglomerativeClustering(n_clusters=optimal_k, linkage=best_linkage)
hierarchical_labels = best_hierarchical.fit_predict(X_hierarchical)

print(f"\nBest Hierarchical Clustering Results (linkage='{best_linkage}'):")
print(f"Silhouette Score: {silhouette_score(X_hierarchical, hierarchical_labels):.4f}")
print(f"Calinski-Harabasz Index: {calinski_harabasz_score(X_hierarchical, hierarchical_labels):.2f}")

# Visualize Hierarchical clusters
X_pca_hier = pca_cluster.transform(X_hierarchical)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
scatter = plt.scatter(X_pca_hier[:, 0], X_pca_hier[:, 1], 
                     c=hierarchical_labels, cmap='tab10', alpha=0.6, s=2)
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('Hierarchical Clustering Results (PCA Visualization)')
plt.colorbar(scatter)

plt.subplot(1, 2, 2)
scatter = plt.scatter(X_pca_hier[:, 0], X_pca_hier[:, 1], 
                     c=pd.Categorical(y_hierarchical).codes, cmap='tab20', alpha=0.6, s=2)
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('True Genre Labels (PCA Visualization)')
plt.colorbar(scatter)

plt.tight_layout()
plt.show()

#### 5.3 Comparações finais

Compare e comente sobre os dois modelos feitos anteriormente. Utilize gráficos para melhor visualização.

In [ ]:
# Compare clustering methods
clustering_comparison = pd.DataFrame({
    'Method': ['K-Means', 'Hierarchical'],
    'Silhouette Score': [
        silhouette_score(X_clustering, kmeans_labels),
        silhouette_score(X_hierarchical, hierarchical_labels)
    ],
    'Calinski-Harabasz Index': [
        calinski_harabasz_score(X_clustering, kmeans_labels),
        calinski_harabasz_score(X_hierarchical, hierarchical_labels)
    ],
    'Sample Size': [len(X_clustering), len(X_hierarchical)]
})

print("Clustering Performance Comparison:")
print(clustering_comparison)

# Visualize comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

clustering_comparison.set_index('Method')['Silhouette Score'].plot(kind='bar', ax=ax1, color='purple')
ax1.set_title('Clustering Method: Silhouette Score Comparison')
ax1.set_ylabel('Silhouette Score')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

clustering_comparison.set_index('Method')['Calinski-Harabasz Index'].plot(kind='bar', ax=ax2, color='teal')
ax2.set_title('Clustering Method: Calinski-Harabasz Index Comparison')
ax2.set_ylabel('Calinski-Harabasz Index')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

- K-Means generally performs better in terms of computational efficiency
- Hierarchical clustering provides more interpretable cluster structure
- Both methods reveal natural groupings in the music feature space
- Cluster quality metrics help validate the clustering approach